In [1]:
import warnings

warnings.filterwarnings('ignore')

########BTCUSDT:binance数据

In [2]:
from jaqs_fxdayu.data.dataservice import LocalDataService
ds = LocalDataService()
from time import time
## 加freq参数

start = time()
path = r'./VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume','symbol': 'BTCUSDT:binance', 'freq': '4H',
         'start_date':20180601000000}

Time_dict = ds.bar_reader(path,props)
print(time()-start)

5.394744157791138


In [3]:
from jaqs_fxdayu.data.hf_dataview import HFDataView
dv = HFDataView()

In [4]:
dv.create_init_dv(Time_dict.set_index(["trade_date","symbol"]))

Initialize dataview success.


WVAD因子

In [5]:
Wvad_j = dv.add_formula('Wvad_j', "Ts_Sum(((close-open)/(high-low))*volume,24)" ,
                             add_data=True)
Ma6wvad_j = dv.add_formula('Ma6wvad_j', 'Ts_Mean(Wvad_j,6)',add_data=True)
Ma24wvad_j = dv.add_formula('Ma24wvad_j', 'Ts_Mean(Wvad_j,24)', add_data=True)

In [6]:
long = dv.add_formula('long',"If(((Ma6wvad_j>=0)&&(Ma6wvad_j>Ma24wvad_j)&&(Delay(Ma6wvad_j,1)<=0))||((Ma24wvad_j>=0)&&(Ma6wvad_j>Ma24wvad_j)&&(Delay(Ma6wvad_j,1)<Delay(Ma24wvad_j,1))), 2, 0)", add_data=True)
short = dv.add_formula('short',"If(((Ma6wvad_j<Ma24wvad_j)&&(Ma6wvad_j<=0)&&(Delay(Ma6wvad_j,1)>=0))||((Ma24wvad_j<=0)&&(Ma6wvad_j<Ma24wvad_j)&&(Delay(Ma6wvad_j,1)>Delay(Ma24wvad_j,1))), -2, 0)", add_data=True)
closeLong = dv.add_formula('closeLong',"If((Ma24wvad_j>=0)&&(Ma6wvad_j<Ma24wvad_j)&&(Delay(Ma6wvad_j,1)>Delay(Ma24wvad_j,1)),1,0)", add_data=True)
closeShort = dv.add_formula("closeShort","If((Ma24wvad_j<=0)&&(Ma6wvad_j>Ma24wvad_j)&&(Delay(Ma6wvad_j,1)<Delay(Ma24wvad_j,1)),-1,0)", add_data=True)

In [7]:
from jaqs_fxdayu.research import TimingDigger

tdMaStatistic = TimingDigger(output_folder=".", output_format='pdf',signal_name='MaStatistic')

def TimingSignal(td, dv, mhp=None,sl=None,sp=None):
    # step 1：实例化TimingDigger 通过output_folder和output_format指定测试报告的输出路径和输出格式，通过signal_name指定绩效文件名称


    #多空分别计算一遍 输出汇总结果
    td.process_signal(
        enter_signal=dv.get_ts("long"),
        exit_signal=dv.get_ts("closeLong"),
        sig_type="long",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

    td.process_signal(
        enter_signal=dv.get_ts("short"),
        exit_signal=dv.get_ts("closeShort"),
        sig_type="short",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )
TimingSignal(tdMaStatistic, dv, 100, 0.05, 0.7)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 2%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 1%


In [8]:
tdMaStatistic.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                      win   loss     all
t-stat              1.520 -4.712  -0.201
p-value             0.189  0.005   0.844
mean                0.036 -0.042  -0.003
std                 0.053  0.020   0.056
info_ratio          0.680 -2.107  -0.061
skewness            1.746 -0.137   1.522
kurtosis            1.123 -0.424   2.440
pct5                0.006 -0.071  -0.063
pct25               0.009 -0.049  -0.042
pct50               0.014 -0.042  -0.002
pct75               0.020 -0.035   0.013
pct95               0.120 -0.016   0.081
occurance           6.000  6.000  12.000
win_ratio             NaN    NaN   0.500
win_mean/loss_mean    NaN    NaN   0.841


In [9]:
tdMaStatistic.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                      win    loss    all
t-stat              3.452 -11.525  2.031
p-value             0.018   0.055  0.082
mean                0.083  -0.036  0.054
std                 0.054   0.003  0.070
info_ratio          1.544 -11.525  0.768
skewness           -0.126  -0.000  0.061
kurtosis           -1.504  -2.000 -1.467
pct5                0.014  -0.039 -0.037
pct25               0.033  -0.038  0.002
pct50               0.088  -0.036  0.049
pct75               0.131  -0.034  0.107
pct95               0.149  -0.033  0.148
occurance           6.000   2.000  8.000
win_ratio             NaN     NaN  0.750
win_mean/loss_mean    NaN     NaN  2.316


In [10]:
tdMaStatistic.create_event_report(sig_type="long_short")#BTCUSDT:binance WVAD因子结果

*****-Summary-*****
Event Analysis
                       win   loss     all
t-stat               3.383 -6.091   1.247
p-value              0.006  0.000   0.227
mean                 0.060 -0.041   0.019
std                  0.058  0.018   0.068
info_ratio           1.020 -2.302   0.286
skewness             0.615 -0.404   0.796
kurtosis            -1.359  0.322  -0.488
pct5                 0.006 -0.068  -0.053
pct25                0.014 -0.045  -0.034
pct50                0.019 -0.041   0.010
pct75                0.107 -0.033   0.037
pct95                0.151 -0.018   0.150
occurance           12.000  8.000  20.000
win_ratio              NaN    NaN   0.600
win_mean/loss_mean     NaN    NaN   1.458


MACD加大阳线（阴线）因子

In [11]:
Diff_J = dv.add_formula('Diff_J',"Ta('EMA',0,0,0,0,close,0,12)-Ta('EMA',0,0,0,0,close,0,26)" , add_data=True)
DEA_J = dv.add_formula('DEA_J', "Ta('EMA',0,0,0,0,Diff_J,0,9)", add_data=True)

In [12]:
MACD=dv.add_formula('MACD','Diff_J-DEA_J', add_data=True)

In [13]:
long = dv.add_formula('long','If(((close/open>1.03)&&(high/low<(close/open+0.02)))&&((MACD>=0)&&(Delay(MACD,1)<=0)),2,0)', add_data=True)
short = dv.add_formula('short','If(((close/open<0.97)&&(high/low<(open/close+0.02)))&&((MACD<=0)&&(Delay(MACD,1)>=0)),-2,0)', add_data=True)
closeLong = dv.add_formula('closeLong','If(((close/open<0.98)&&(high/low<(open/close+0.02)))&&((MACD<=0)&&(Delay(MACD,1)>=0)),1,0)', add_data=True)
closeShort = dv.add_formula("closeShort",'If(((close/open>1.02)&&(high/low<(close/open+0.02)))&&((MACD>=0)&&(Delay(MACD,1)<=0)),-1,0)', add_data=True)

Field [long] is overwritten.
Field [short] is overwritten.
Field [closeLong] is overwritten.
Field [closeShort] is overwritten.


In [14]:
from jaqs_fxdayu.research import TimingDigger

tdMaStatistic = TimingDigger(output_folder=".", output_format='pdf',signal_name='MaStatistic')

def TimingSignal(td, dv, mhp=None,sl=None,sp=None):
    # step 1：实例化TimingDigger 通过output_folder和output_format指定测试报告的输出路径和输出格式，通过signal_name指定绩效文件名称


    #多空分别计算一遍 输出汇总结果
    td.process_signal(
        enter_signal=dv.get_ts("long"),
        exit_signal=dv.get_ts("closeLong"),
        sig_type="long",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

    td.process_signal(
        enter_signal=dv.get_ts("short"),
        exit_signal=dv.get_ts("closeShort"),
        sig_type="short",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )
TimingSignal(tdMaStatistic, dv, 100, 0.1, 0.7)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 0%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 1%


In [15]:
tdMaStatistic.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                      win  loss    all
t-stat              1.529   NaN  1.529
p-value             0.369   NaN  0.369
mean                0.139   NaN  0.139
std                 0.091   NaN  0.091
info_ratio          1.529   NaN  1.529
skewness            0.000   NaN  0.000
kurtosis           -2.000   NaN -2.000
pct5                0.057   NaN  0.057
pct25               0.093   NaN  0.093
pct50               0.139   NaN  0.139
pct75               0.184   NaN  0.184
pct95               0.220   NaN  0.220
occurance           2.000   NaN  2.000
win_ratio             NaN   NaN  1.000
win_mean/loss_mean    NaN   NaN    NaN


In [16]:
tdMaStatistic.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                       win   loss    all
t-stat              11.066 -5.730 -0.110
p-value              0.057  0.110  0.920
mean                 0.088 -0.100 -0.006
std                  0.008  0.017  0.095
info_ratio          11.066 -5.730 -0.063
skewness             0.000  0.000 -0.040
kurtosis            -2.000 -2.000 -1.920
pct5                 0.081 -0.115 -0.112
pct25                0.084 -0.108 -0.091
pct50                0.088 -0.100 -0.001
pct75                0.092 -0.091  0.084
pct95                0.095 -0.084  0.093
occurance            2.000  2.000  4.000
win_ratio              NaN    NaN  0.500
win_mean/loss_mean     NaN    NaN  0.880


In [17]:
tdMaStatistic.create_event_report(sig_type="long_short") #BTCUSDT:binance MACD因子结果

*****-Summary-*****
Event Analysis
                      win   loss    all
t-stat              2.831 -5.730  0.817
p-value             0.066  0.110  0.451
mean                0.113 -0.100  0.042
std                 0.069  0.017  0.116
info_ratio          1.634 -5.730  0.365
skewness            0.940  0.000  0.085
kurtosis           -0.807 -2.000 -1.021
pct5                0.053 -0.115 -0.108
pct25               0.072 -0.108 -0.050
pct50               0.088 -0.100  0.064
pct75               0.129 -0.091  0.092
pct95               0.209 -0.084  0.196
occurance           4.000  2.000  6.000
win_ratio             NaN    NaN  0.667
win_mean/loss_mean    NaN    NaN  1.135


#############EOSUSDT:binance数据

In [18]:
from jaqs_fxdayu.data.dataservice import LocalDataService
ds = LocalDataService()
from time import time
## 加freq参数

start = time()
path = r'./VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume','symbol': 'EOSUSDT:binance', 'freq': '4H',
         'start_date':20180601000000}

Time_dict = ds.bar_reader(path,props)

In [19]:
dv.create_init_dv(Time_dict.set_index(["trade_date","symbol"]))

Initialize dataview success.


WVAD因子

In [20]:
Wvad_j = dv.add_formula('Wvad_j', "Ts_Sum(((close-open)/(high-low))*volume,24)" ,
                             add_data=True)
Ma6wvad_j = dv.add_formula('Ma6wvad_j', 'Ts_Mean(Wvad_j,6)',add_data=True)
Ma24wvad_j = dv.add_formula('Ma24wvad_j', 'Ts_Mean(Wvad_j,24)', add_data=True)

In [21]:
long = dv.add_formula('long',"If(((Ma24wvad_j>=0)&&(Ma6wvad_j>Ma24wvad_j)&&(Delay(Ma6wvad_j,1)<Delay(Ma24wvad_j,1))), 2, 0)", add_data=True)
short = dv.add_formula('short',"If((Ma24wvad_j<=0)&&((Ma6wvad_j<Ma24wvad_j)&&(Delay(Ma6wvad_j,1)>Delay(Ma24wvad_j,1))), -2, 0)", add_data=True)
closeLong = dv.add_formula('closeLong',"If(short==-2,1,0)", add_data=True)
closeShort = dv.add_formula("closeShort","If(long==2,-1,0)", add_data=True)

In [22]:
from jaqs_fxdayu.research import TimingDigger

tdMaStatistic = TimingDigger(output_folder=".", output_format='pdf',signal_name='MaStatistic')

def TimingSignal(td, dv, mhp=None,sl=None,sp=None):
    # step 1：实例化TimingDigger 通过output_folder和output_format指定测试报告的输出路径和输出格式，通过signal_name指定绩效文件名称


    #多空分别计算一遍 输出汇总结果
    td.process_signal(
        enter_signal=dv.get_ts("long"),
        exit_signal=dv.get_ts("closeLong"),
        sig_type="long",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

    td.process_signal(
        enter_signal=dv.get_ts("short"),
        exit_signal=dv.get_ts("closeShort"),
        sig_type="short",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )
TimingSignal(tdMaStatistic, dv, 100, 0.05, 0.7)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 0%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 1%


In [23]:
tdMaStatistic.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                    win      loss       all
t-stat              NaN       NaN       NaN
p-value             NaN       NaN       NaN
mean                NaN -0.108000 -0.108000
std                 NaN  0.000000  0.000000
info_ratio          NaN      -inf      -inf
skewness            NaN  0.000000  0.000000
kurtosis            NaN -3.000000 -3.000000
pct5                NaN -0.108000 -0.108000
pct25               NaN -0.108000 -0.108000
pct50               NaN -0.108000 -0.108000
pct75               NaN -0.108000 -0.108000
pct95               NaN -0.108000 -0.108000
occurance           NaN  1.000000  1.000000
win_ratio           NaN       NaN       NaN
win_mean/loss_mean  NaN       NaN       NaN


In [24]:
tdMaStatistic.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                      win      loss    all
t-stat              7.317       NaN  3.130
p-value             0.002       NaN  0.026
mean                0.157 -0.052000  0.122
std                 0.043  0.000000  0.087
info_ratio          3.658      -inf  1.400
skewness           -0.720  0.000000 -1.106
kurtosis           -0.477 -3.000000 -0.121
pct5                0.095 -0.052000 -0.019
pct25               0.151 -0.052000  0.098
pct50               0.167 -0.052000  0.159
pct75               0.175 -0.052000  0.173
pct95               0.203 -0.052000  0.202
occurance           5.000  1.000000  6.000
win_ratio             NaN       NaN  0.833
win_mean/loss_mean    NaN       NaN  3.009


In [25]:
tdMaStatistic.create_event_report(sig_type="long_short")#EOSUSDT:binance WVAD因子结果

*****-Summary-*****
Event Analysis
                      win   loss    all
t-stat              7.317 -2.849  1.913
p-value             0.002  0.215  0.104
mean                0.157 -0.080  0.089
std                 0.043  0.028  0.114
info_ratio          3.658 -2.849  0.781
skewness           -0.720 -0.000 -0.711
kurtosis           -0.477 -2.000 -1.107
pct5                0.095 -0.106 -0.092
pct25               0.151 -0.094  0.014
pct50               0.167 -0.080  0.151
pct75               0.175 -0.066  0.171
pct95               0.203 -0.055  0.200
occurance           5.000  2.000  7.000
win_ratio             NaN    NaN  0.714
win_mean/loss_mean    NaN    NaN  1.953


In [26]:
Diff_J = dv.add_formula('Diff_J',"Ta('EMA',0,0,0,0,close,0,12)-Ta('EMA',0,0,0,0,close,0,26)" , add_data=True)
DEA_J = dv.add_formula('DEA_J', "Ta('EMA',0,0,0,0,Diff_J,0,9)", add_data=True)

In [27]:
MACD=dv.add_formula('MACD','Diff_J-DEA_J', add_data=True)

In [28]:
long = dv.add_formula('long','If(((close/open>1.03)&&(high/low<(close/open+0.02)))&&((MACD>=0)&&(Delay(MACD,1)<=0)),2,0)', add_data=True)
short = dv.add_formula('short','If(((close/open<0.98)&&(high/low<(open/close+0.02)))&&((MACD<=0)&&(Delay(MACD,1)>=0)),-2,0)', add_data=True)
closeLong = dv.add_formula('closeLong','If(((close/open<0.98)&&(high/low<(open/close+0.02)))&&((MACD<=0)&&(Delay(MACD,1)>=0)),1,0)', add_data=True)
closeShort = dv.add_formula("closeShort",'If(((close/open>1.02)&&(high/low<(close/open+0.02)))&&((MACD>=0)&&(Delay(MACD,1)<=0)),-1,0)', add_data=True)

Field [long] is overwritten.
Field [short] is overwritten.
Field [closeLong] is overwritten.
Field [closeShort] is overwritten.


In [29]:
from jaqs_fxdayu.research import TimingDigger

tdMaStatistic = TimingDigger(output_folder=".", output_format='pdf',signal_name='MaStatistic')

def TimingSignal(td, dv, mhp=None,sl=None,sp=None):
    # step 1：实例化TimingDigger 通过output_folder和output_format指定测试报告的输出路径和输出格式，通过signal_name指定绩效文件名称


    #多空分别计算一遍 输出汇总结果
    td.process_signal(
        enter_signal=dv.get_ts("long"),
        exit_signal=dv.get_ts("closeLong"),
        sig_type="long",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

    td.process_signal(
        enter_signal=dv.get_ts("short"),
        exit_signal=dv.get_ts("closeShort"),
        sig_type="short",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )
TimingSignal(tdMaStatistic, dv, 100, 0.05, 0.7)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 0%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 1%


In [30]:
tdMaStatistic.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                    win      loss       all
t-stat              NaN       NaN       NaN
p-value             NaN       NaN       NaN
mean                NaN -0.070000 -0.070000
std                 NaN  0.000000  0.000000
info_ratio          NaN      -inf      -inf
skewness            NaN  0.000000  0.000000
kurtosis            NaN -3.000000 -3.000000
pct5                NaN -0.070000 -0.070000
pct25               NaN -0.070000 -0.070000
pct50               NaN -0.070000 -0.070000
pct75               NaN -0.070000 -0.070000
pct95               NaN -0.070000 -0.070000
occurance           NaN  1.000000  1.000000
win_ratio           NaN       NaN       NaN
win_mean/loss_mean  NaN       NaN       NaN


In [31]:
tdMaStatistic.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                      win   loss    all
t-stat              2.445 -4.886  1.177
p-value             0.058  0.039  0.273
mean                0.137 -0.088  0.062
std                 0.125  0.026  0.148
info_ratio          1.093 -3.455  0.416
skewness            1.326 -0.046  0.965
kurtosis            0.402 -1.500  0.428
pct5                0.028 -0.117 -0.107
pct25               0.052 -0.104 -0.058
pct50               0.109 -0.088  0.042
pct75               0.138 -0.073  0.137
pct95               0.333 -0.061  0.294
occurance           6.000  3.000  9.000
win_ratio             NaN    NaN  0.667
win_mean/loss_mean    NaN    NaN  1.544


In [32]:
tdMaStatistic.create_event_report(sig_type="long_short")#EOSUSDT:binance MACD因子结果

*****-Summary-*****
Event Analysis
                      win   loss     all
t-stat              2.445 -6.172   0.996
p-value             0.058  0.009   0.345
mean                0.137 -0.084   0.048
std                 0.125  0.024   0.146
info_ratio          1.093 -3.563   0.332
skewness            1.326 -0.517   1.105
kurtosis            0.402 -1.161   0.686
pct5                0.028 -0.115  -0.106
pct25               0.052 -0.096  -0.067
pct50               0.109 -0.079   0.033
pct75               0.138 -0.067   0.123
pct95               0.333 -0.059   0.281
occurance           6.000  4.000  10.000
win_ratio             NaN    NaN   0.600
win_mean/loss_mean    NaN    NaN   1.628
